<a href="https://colab.research.google.com/github/yue-sun/generative-art/blob/d3_cell_automata/03_wednesday/01_cellular_automata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cellular automata

## Cellular automata in 1D
A cellular automaton is a system of interacting cells on a grid whose future state depends on the states of neighboring cells according to a pre-defined rule. Let's start by looking at cellular automata in one dimension. The following represents a sequence of cells:
![cells](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/cells.png)

The state of each cell is represented by a binary value, and in 1D, each cell has two neighbors. Thus, we can look at the "code" formed by considering a cell and its two neighbors in the order left, center, right (LCR).

![triplets](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/ca_triplet.png)

This binary triplet, such as 011 in the example shown, can be thought of as a unique identifier for that specific neighbor state configuration. We can also convert the representation of the code from binary to decimal.

If our codes are only 3 digits long in binary, how many possible combinations are there? There are $2\times2\times2=8$ possibilities. Thus, our rules for the evolution of cellular automata must prescribe an outcome for each of the 8 possible configurations we can observe. These rules were formalized and extensively studied by Wolfram. Each rule is expressed as an 8-bit binary string, such as the example Rule 30 below:

![rules](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/wolfram_rules.png)

For each of the 8 neighbor configurations, the rule defines the state of the **center cell** in the next time step.

We'll begin by writing a function that allows us to simulate cellular automata in one dimension. First, let's implement a helper function that allows us to generate the 8-bit rule representations.

In [ ]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
import ipywidgets as widgets
import gc
import matplotlib.animation as animation
from IPython.display import HTML
from skimage.io import imread
from google.colab import files

In [ ]:
# Use numpy's decimal to binary representation function, binary_repr.
# The first value is the decimal number, and the second is the number
# of digits in binary to return.
np.binary_repr(2,8)

In [ ]:
def get_rule(n, show=True):
    '''Get the binary representation of a rule. Valid codes are n=0 to n=255.'''
    if n<0 or n>255:
        raise ValueError("Rule index must be between 0 and 255.")
        
    rule = np.array([int(i) for i in np.binary_repr(n,8)], dtype=np.uint8)
    # flip the order of the entries so that rule[decimal_rep] = new_state,
    # where decimal_rep is the decimal representation of the 3-bit neighborhood.
    rule = rule[::-1]
    if show:
        plot_rule(rule)
    return rule

def plot_rule(rule):
    print("Rule",sum([rule[i]*2**i for i in range(len(rule))]))
    fig, axes = plt.subplots(2,8,figsize=(8,2))
    
    for i in range(8):   # iterate over all possible neighborhoods
        neigh = np.array([int(j) for j in np.binary_repr(i,3)])    # convert to binary
        new_state = np.nan*np.ones((3,3)); new_state[1,1] = rule[i]
        ind = 7-i # facilitate plotting from right to left.
        
        axes[0,ind].imshow(neigh.reshape(1,3), vmin=0, vmax=1)     # plot the binary triplet
        axes[1,ind].imshow(new_state, vmin=0, vmax=1)              # plot the new cell state below
        axes[0,ind].axis('off'); axes[1,ind].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
rule = get_rule(2)

## Implementation

There are several different ways to implement the update for cellular automata, which applies a specified rule to a set of cells. Here, we present an approach using **convolution**. A visual representation of discrete convolution is illustrated below:

![convolution](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/convolution.png)

A convolution **kernel** scans across an array, here representing the cell states. At each position, the coinciding values of the kernel and array are multiplied, then added together. If the operation is performed cyclically, such that the kernel wraps back to the beginning, the result is a new array of the same size as the original, whose values have been transformed by the kernel.

In the context of cellular automata, we can use convolution to perform binary-to-decimal conversion, by choosing the kernel $[2^0,2^1,2^2]=[1,2,4]$. Thus, we convert all triplet states to their decimal representation, an integer from 0 to 7. Then, we can use this decimal code to find the appropriate next state based on our chosen rule.

![bin2dec](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/bin_to_dec.png)

The function ```cellular_automaton``` below performs a simulation of a cellular automaton given a rule, the number of discrete steps to advance in time, and the initial states of the cells. Internally, the ```step``` function applies a 1D convolution to convert the current cell states to their decimal representations, then determine the next states based on the supplied rule.


In [ ]:
from scipy.ndimage import convolve1d

def cellular_automaton(rule, steps, init):
    """Simulate a cellular automaton.
    rule - the 8-bit representation of the rule.
    steps - the number of iterations to perform.
    init - the initial state of the cells.
    """
    size = len(init) # get the number of cells.
    cells = np.zeros((steps+1, size), dtype=np.uint8)
    cells[0,:] = init
    
    def step(cells, rule):
        # convolve cells with "converter" array to convert binary LCR triplets
        # to base 10 integers from 0 to 7.
        bin2dec = np.array([1,2,4], dtype=int)
        decimal_rep = convolve1d(cells, bin2dec, mode='wrap')

        # Get updated patterns from the rule for all cells.
        return rule[decimal_rep]
    
    # Apply the step function iteratively.
    for i in range(steps):
        cells[i+1,:] = step(cells[i,:], rule)
    return cells

We also define helper functions to plot and animate the cell states:

In [ ]:
# simple function to plot states.
def plot_cells(cells, figsize=(9,9)):
    plt.figure(figsize=figsize)
    plt.imshow(cells, interpolation='nearest')
    plt.axis('off')
    plt.show()

#generate a sequence of plots assembled into an animation.
def make_animation1d(cells, vmin=0, vmax=1, cmap=plt.cm.viridis):
    frames = len(cells)
    fig, ax = plt.subplots(1,1,figsize=(6,6))
    plot_cells = np.zeros_like(cells)
    plot_cells[0] = cells[0]
    im = ax.imshow(plot_cells, vmin=vmin, vmax=vmax, interpolation='nearest', cmap=cmap)
    ax.axis('off'); fig.tight_layout()

    def animate(i):
        '''Plot updates for animation.'''
        plot_cells[i] = cells[i]
        im.set_array(plot_cells)
        return im,

    ani = animation.FuncAnimation(fig, animate, frames=frames, interval=64, blit=True)
    plt.close(fig)
    return ani

Below, we test our implementation on a system of 201 cells following Rule 18. We start by only turning on the center-most cell as our initial condition.

In [ ]:
n = 18 # rule to follow
rule = get_rule(n)

In [ ]:
N = 201 # number of cells
T = 100  # number of time steps

# set the initial condition to a single "on" cell in the middle.
init = np.zeros(N, dtype=np.uint8); init[N//2] = 1
cells = cellular_automaton(rule, T, init)

# create animation
ani = make_animation1d(cells)

# display the animation
HTML(ani.to_html5_video())

We can also randomly initialize the cell states:

In [ ]:
n = 30 # rule to follow
rule = get_rule(n)

N = 201 # number of cells
T = 100  # number of time steps

# set a random initial condition.
np.random.seed(12)
init = (np.random.random(N)>0.5).astype(np.uint8)
cells = cellular_automaton(rule, T, init)

# create animation
ani = make_animation1d(cells)

# display the animation
HTML(ani.to_html5_video())

[Rule 30](https://en.wikipedia.org/wiki/Rule_30) is a notable one due to its striking resemblance to biological shell patterns (See [here](https://www.wolframscience.com/nks/p423--biological-pigmentation-patterns/) for more examples)! Try and experiment with different rules: How many possible rules are there in 1D?

## Cellular Automata in 2D
The principle of update rules based on nearest-neighbor states can also be extended to 2D. Two types of neighborhoods are commonly considered: a 5-cell (von Neumann) and a 9-cell (Moore) neighboorhood.
![neighborhoods](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/2d_neigh.png)

Here we will consider a two examples of cellular automata in 2D: Conway's Game of Life, and diffusion-limited aggregation.

## Conway's Game of Life

Conway's Game of Life is a 2D cellular automaton that obeys the following rules for updating cell states:
1. A live cell with less than two or more than three live neighbors dies in the next iteration (under and overpopulation effects).
2. A dead cell with exactly three live neighbors becomes a live cell in the next iteration (reproduction).
3. All other cell states remain unchanged.

As the system evolves from an initial state, it can exhibit a wide variety of motifs, some of which occur frequently. Some common patterns are **still lifes**, which do not change in the next iteration, **oscillators**, which are periodic after some number of iterations, and **spaceships**, which translate in space. A summary of frequently occurring patterns can be found [here](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life#Examples_of_patterns).

The function below implements the rules for Conway's Game of Life. Again, we'll make use of convolution to efficiently count the number of live cells in each cell neighborhood. The following convolution kernel allows us to compute the sum of all neighboring cell states, therefore giving us the number of neighboring live cells:

$$
\begin{pmatrix}
1 & 1 & 1 \\
1 & 0 & 1 \\
1 & 1 & 1
\end{pmatrix}
$$

Though different from the traditional Game of Life, here we will also use periodic boundary conditions to keep our cells bounded.


In [ ]:
from scipy.signal import convolve2d

def game_of_life(steps, init):
    """Simulate Conway's game of life.
    steps - the number of iterations to perform.
    init - the initial state of the cells.
    """
    cells = np.zeros((steps+1, init.shape[0], init.shape[1]), dtype=np.uint8)
    cells[0] = np.copy(init)

    # Set up convolution kernel for summing up neighbor states in a Moore neighborhood,
    # excluding the center cell's state.
    kernel = np.ones((3,3), dtype=int); kernel[1,1] = 0
    
    def step(cells_prev, cells):
        # copy the current cell states
        cells[:] = cells_prev[:]

        # Use convolution to compute the neighbor sums in a Moore neighborhood
        
        neigh_sum = convolve2d(cells_prev, kernel,
                                 mode = 'same',
                                 boundary = 'wrap') # periodic boundaries
        # Apply rules:
        # 1. A live cell with exactly two or three live neighbours survives
        # (others die due to over or under population).
        # 2. A dead cell with exactly three live neighbours becomes a live cell (reproduction).
        cells[(cells_prev == 1)&((neigh_sum < 2)|(neigh_sum > 3))] = 0
        cells[(cells_prev == 0)&(neigh_sum == 3)] = 1
        
    # Apply the step function iteratively.
    for i in range(steps):
        step(cells[i], cells[i+1])
    return cells

Let's generate a random initial state for our game. In order to avoid too many single live cells with no neighbors, which disappear after one iteration, we use the ```binary_blobs``` library function to produce some larger cell clusters.

In [ ]:
from skimage.data import binary_blobs

N = 101
T = 200

init = binary_blobs(N, blob_size_fraction=5./N, volume_fraction=5./N)
plot_cells(init)

Let's run the simulation!

In [ ]:
# run the simulation and visualize the last frame
cells = game_of_life(T, init)
plot_cells(cells[-1]) # plot the last frame

Next we add an animation routine to visualize the evolution of the system:

In [ ]:
#generate a sequence of plots assembled into an animation.
def make_animation2d(cells, vmin=0, vmax=1, cmap=plt.cm.viridis, skip=1):
    frames = len(cells)//skip
    fig, ax = plt.subplots(1,1,figsize=(6,6))
    im = ax.imshow(cells[0], vmin=vmin, vmax=vmax, interpolation='nearest', cmap=cmap)
    ax.axis('off'); fig.tight_layout()

    def animate(i):
        '''Plot updates for animation.'''
        im.set_array(cells[i*skip])
        return im,

    ani = animation.FuncAnimation(fig, animate, frames=frames, interval=64, blit=True)
    plt.close(fig)
    return ani

In [ ]:
# create animation
ani = make_animation2d(cells)

# display the animation
HTML(ani.to_html5_video())

### Methuselahs

Methuselahs are initial seed patterns in Conway's Game of Life that take a long time to stabilize (or reach a dynamic steady state). A few examples are provided as images on the course website that can be loaded in to initialize the simulation. You can find a list of notable methuselahs [here](https://conwaylife.com/wiki/List_of_long-lived_methuselahs)!

In [ ]:
# path to test images on course website
path = "https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/methuselahs/"

init = imread(path+'queen_bee_gliders.png', as_gray=True).astype(np.uint8)
#init = imread(path+'thunderbird.png', as_gray=True).astype(np.uint8)
#init = imread(path+'pi_heptomino.png', as_gray=True).astype(np.uint8)
plot_cells(init)

In [ ]:
T = 300
cells = game_of_life(T, init)

# create animation
ani = make_animation2d(cells)

# display the animation
HTML(ani.to_html5_video())

## Diffusion-Limited Aggregation
The final application of cellular automata we'll consider here is diffusion-limited aggregation (DLA). This model dates back to 1981 in a [paper by Witten and Sander](https://ifisc.uib-csic.es/~tomas/MFCS/EdenDLA/PhysRevLett.47.1400.pdf), who proposed the model as a way to study dendritic structures of metal particles. In DLA, we imagine there are particles housed on a grid of cells. Each particle may be mobile or fixed, and each grid cell may be empty or occupied. When mobile particles come into contact with a fixed particle, they stick! We thus use three colors to refer to three different possible cell states: empty cell (0), cell with mobile particle (1), cell with fixed particle (2). DLA can be used to model processes like crystal growth through the diffusion of ions or molecules, or the condensation of water vapor to form snowflakes. The tree-like structures that develop in DLA systems also have fractal characteristics.

![dla_grid](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/dla_grid.png)

The rules are as follows:
1. If a mobile particle neighbors at least one fixed particle within its Moore neighborhood, it becomes fixed.
2. If a mobile particle does not neighbor a fixed particle, it moves to an empty cell in its neighborhood, chosen at random. If no empty cells are available, the mobile particle stays in its current position.

![dla_rules](https://raw.githubusercontent.com/yue-sun/generative-art/d3_cell_automata/03_wednesday/figs/dla_rules.png)

In contrast to the 1D cellular automata and Conway's Game of Life, these rules are *not* simultaneous: Each particle is updated one at a time. That is, when one particle moves or becomes fixed, subsequent particles must refer to the new position and state of that particle.

We will use two separate data structures to keep track of the particles and cells. We'll create an $N\times 2$ array that stores the $(j,k)$ index of each particle's location:

$$
\begin{matrix}
\text{particle 1} \\
\text{particle 2} \\
\vdots \\
\text{particle }N
\end{matrix}
\begin{bmatrix}
j_1 & k_1 \\
j_2 & k_2 \\
\vdots & \vdots \\
j_N & k_N
\end{bmatrix},
$$

and an $m\times n$ grid of cells storing the cell states: empty (0), occupied by a mobile particle (1), or occupied by a fixed particle (2):

$$
\begin{bmatrix}
0 & 1 & 0 & \cdots & 0 & 0 & 1 \\
1 & 0 & 0 & \cdots & 2 & 1 & 0 \\
& & & \vdots & & & \\
1 & 2 & 2 & \cdots & 1 & 0 & 0
\end{bmatrix}.
$$

The particle array allows us to quickly iterate over only the cells containing particles. The cell grid then allows us to retrieve the type of particle (mobile or fixed) and determine its neighbors.

Let's start by defining a function to initialize an $m\times n$ grid and populate it with $N$ particles, $N_f$ of which are fixed.



In [ ]:
def initialize_dla(m, n, N, Nf=1, seed=12, proba=None):
    """Initialize a grid and particles for diffusion-limited aggregation.
    m, n - grid dimensions (rows and columns).
    N - number of total particles, must be smaller than m times n.
    Nf - number of initial fixed particles.
    seed - random seed.
    proba - an m by n array of the relative probability with which each grid cell
            should be chosen, for the mobile particles.
    """
    if N > m*n:
        raise ValueError("Number of particles must be smaller than m times n.")

    if proba is None:
        # Each grid cell is equally likely to be chosen.
        proba=np.ones((m,n))
    proba = proba.ravel() # reshape into linear array.

    np.random.seed(seed)

    # Enumerate all possible particle locations (j,k).
    spots = np.array([index_pair for index_pair in np.ndindex(m, n)])

    # Select the positions of the Nf fixed particles.
    # If Nf = 1, reserve the center spot (m//2,n//2) for the initial particle.
    if Nf == 1:
        fixed_spots = spots[n*(m//2)+(n//2)]
        spots = np.delete(spots, n*(m//2)+(n//2), axis=0) # remove chosen spot from selection
        proba = np.delete(proba, n*(m//2)+(n//2), axis=0)
    
    # Otherwise, initialize Nf seed particles randomly.
    else:
        fixed_indices = np.random.choice(m*n, size=Nf, replace=False) # choose Nf spots at random
        fixed_spots = spots[fixed_indices]
        spots = np.delete(spots, fixed_indices, axis=0) # remove chosen spots from selection
        proba = np.delete(proba, fixed_indices, axis=0)
    
    # Randomly initialize the N-Nf mobile particles on the remaining spots.
    mobile_indices = np.random.choice(m*n-Nf, size=N-Nf, replace=False, p=proba/np.sum(proba))
    mobile_spots = spots[mobile_indices]

    # Construct the particles array.
    particles = np.zeros((N,2), dtype=int)
    particles[:Nf] = fixed_spots
    particles[Nf:] = mobile_spots

    # Construct and populate the grid of cell states.
    # (0=empty, 1=mobile particle, 2=fixed particle)
    cells = np.zeros((m,n), dtype=np.uint8)
    for i in range(N):
        j,k = particles[i]
        if i < Nf:
            cells[j,k] = 2 # fixed particles
        else:
            cells[j,k] = 1 # mobile particles

    return particles, cells # return the initial state of our two data structures


In [ ]:
m, n = 101, 101
N, Nf = 3000, 1
seed = 12
particles, init = initialize_dla(m, n, N, Nf=Nf, seed=seed)
plot_cells(init)

Finally, we define a routine to simulate diffusion-limited aggregation according to the defined rules.

In [ ]:
def diffusion_limited_aggregation(particles, init, T, seed=12):
    """Simulate diffusion-limited-aggregation.
    particles - array of initial particle positions (j,k).
    init - initial grid of cell states.
    T - number of time steps.
    seed - random seed.
    """
    np.random.seed(seed)

    # Set up grid for tracking cell states in time, and fill the initial state.
    m, n = init.shape
    cells = np.zeros((T+1,m,n), dtype=np.uint8)
    cells[0] = np.copy(init)

    # Particle index list to shuffle for random traversal order each time.
    N = len(particles)
    idx = np.arange(N).astype(int)
    
    @njit
    def step(cells_prev, cells, particles, idx):
        # copy the current cell states
        cells[:] = cells_prev[:]

        # loop over all particles
        for i in idx:
            j,k = particles[i]
            neigh_spots = np.array([((j-1)%m,k), (j,(k-1)%n),
                                    ((j+1)%m,k), (j,(k+1)%n),
                                    ((j-1)%m,(k-1)%n), ((j-1)%m,(k+1)%n),
                                    ((j+1)%m,(k-1)%n), ((j+1)%m,(k+1)%n)]) # Moore neighbors

            # If a fixed particle, particle stays fixed (nothing to do).
            # Only need to take action if mobile particle:
            if cells[j,k] == 1:
                # check the neighbors for any fixed cells
                neighs = np.array([cells[spot[0],spot[1]] for spot in neigh_spots])
                
                # if at least one neighbor is fixed, particle becomes fixed
                if np.any(neighs == 2):
                    cells[j,k] = 2

                # otherwise, randomly move the particle into an empty neighboring cell,
                # if an empty cell exists.
                else:
                    if np.any(neighs == 0):
                        empty_spots = neigh_spots[neighs == 0]
                        particles[i] = empty_spots[np.random.choice(len(empty_spots), size=1)]
                        cells[j,k] = 0
                        p, q = particles[i]
                        cells[p,q] = 1

    # Apply the step function iteratively.
    for i in range(T):
        # loop through particles in random order each time
        np.random.shuffle(idx)
        step(cells[i], cells[i+1], particles, idx)
        
    return cells

In [ ]:
m, n = 301, 301
N, Nf = 15000, 5
seed = 0

particles, init = initialize_dla(m, n, N, Nf=Nf, seed=seed)

plot_cells(init)

T = 1000
cells = diffusion_limited_aggregation(particles, init, T, seed=seed)

In [ ]:
# create animation
ani = make_animation2d(cells, vmin=0, vmax=2, skip=5) # skip=5 animates every 5th frame

# display the animation
HTML(ani.to_html5_video())

We can also only color cells with fixed particles, and vary the color with the time at which cells become incorporated.

In [ ]:
fixed_cells = (cells == 2).astype(float)

for i in range(1,T+1):
    fixed_cells[i] *= (T-i)/T
    fixed_cells[i][fixed_cells[i-1] > 0] = fixed_cells[i-1][fixed_cells[i-1] > 0]

In [ ]:
# create animation
ani = make_animation2d(fixed_cells, vmin=0, vmax=1, skip=5)

# display the animation
HTML(ani.to_html5_video())

Play around with the simulation - how do the branching patterns change when there is a higher or lower density of particles?

# Seeding with an image

Let's vary the distribution of mobile particles across the grid by seeding our initial state with an image! A set of sample images is provided on the course website:

In [ ]:
# path to test images on course website
path = "https://raw.githubusercontent.com/yue-sun/generative-art/main/03_wednesday/figs/seeds/"

im = imread(path+"dragon.png", as_gray=True)
m, n = im.shape
print("Image dimensions:", m, n)

fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(im, interpolation='nearest')
ax.axis('equal')
ax.axis('off')
plt.show()

Suppose we'd like the cells inside our shape to be three times more likely to be selected as the location of a mobile particle in the initial configuration. Let's define an array of the same size as our image that encodes these odds:

In [ ]:
proba = np.ones(im.shape)
proba[im < 0.5] = 3

Initialize the cell states with the new sampling probabilities:

In [ ]:
seed = 0
particles, init = initialize_dla(m, n, 15000, Nf=1, seed=seed, proba=proba)

plot_cells(init)

In [ ]:
T = 1000
cells = diffusion_limited_aggregation(particles, init, T, seed=seed)
plot_cells(cells[-1])

In [ ]:
# create animation
ani = make_animation2d(cells, vmin=0, vmax=2, skip=5)

# display the animation
HTML(ani.to_html5_video())

In [ ]:
fixed_cells = (cells == 2).astype(float)

for i in range(1,T+1):
    fixed_cells[i] *= (T-i)/T
    fixed_cells[i][fixed_cells[i-1] > 0] = fixed_cells[i-1][fixed_cells[i-1] > 0]

# create animation
ani = make_animation2d(fixed_cells, vmin=0, vmax=1, skip=5)

# display the animation
HTML(ani.to_html5_video())

## References
- 1D Cellular automata: http://mathworld.wolfram.com/ElementaryCellularAutomaton.html
- Conway's Game of Life: https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life
- Diffusion-Limited Aggregation: Witten Jr, T. A., & Sander, L. M. (1981). Diffusion-limited aggregation, a kinetic critical phenomenon. Physical review letters, 47(19), 1400. https://ifisc.uib-csic.es/~tomas/MFCS/EdenDLA/PhysRevLett.47.1400.pdf